<a href="https://colab.research.google.com/github/alexandreblima/IC-THS/blob/master/mnist_1_sgd_epoch50_batch128_4layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# arquivo: mnist_1_sgd_epoch50_batch128_4layers.py
# Adaptação dos códigos do livro "Deep Learning with TensorFlow2
# and Keras: Regression, Convnets, GANs RNNs, NLP and more with
# TensorFlow2 and the Keras API, de Antonio Gulli, Amita Kapoor 
# e Sujit Pal, 2a ed., Packt. 
# otimizador = SGD
# código para TensorFlow 2

import tensorflow as tf
#import numpy as np
from tensorflow import keras
print("Tensorflow version " + tf.__version__)
import datetime

%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

# Parâmetros da rede e de treinamento
epocas = 50   # define a duração do treinamento
lote_tam = 128 # número de amostras que alimentarão a rede em uma dada época
               # de treinamento (em inglês, lote = batch)
verbose = 1
n_classes = 10 # número de saídas = número de dígitos
n_oculta = 128 
val_split = 0.2 # a fração do número de amostras de treinamento reservadas para
                # validação => 48.000 amostras para treino + 12.000 amostras para 
                # validação = 60.000 exemplos do conjunto de treinamento MNIST     

# carrega a base de dados MNIST
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

reformat = 784 # é a dimensão do espaço de "features"
               # ou seja, a camada de entrada possui 784 "input units"
               # Uma entrada é um vetor coluna 784 x 1
 
X_train = X_train.reshape(60000, reformat) 
X_test = X_test.reshape(10000, reformat) 

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalização em [0,1]
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'amostras de treinamento')
print(X_test.shape[0], 'amostras de teste')

# Representação "one-hot" dos rótulos (labels) de treinamento e teste
Y_train = tf.keras.utils.to_categorical(Y_train, n_classes)
Y_test = tf.keras.utils.to_categorical(Y_test, n_classes)

# construção do modelo
# A camada de saída é composta por 10 neurônios com função de ativação "softmax", que é uma 
# generalização da função sigmóide ou logística. 

model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(n_oculta, input_shape=(reformat,), name='dense_layer1', activation='relu'))
model.add(keras.layers.Dense(n_oculta, input_shape=(reformat,), name='dense_layer2', activation='relu'))
model.add(keras.layers.Dense(n_classes, input_shape=(reformat,), name='dense_layer3', activation='softmax'))

# sumário do modelo
model.summary()

# compilando o modelo
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True)

# treinamento do modelo no TensorFlow 2
treino = model.fit(X_train, Y_train, batch_size=lote_tam, epochs=epocas, verbose=verbose, validation_split=val_split, callbacks=tensorboard_callback)
# model.fit retorna o objeto "treino", que é do tipo "History".  

treino_dic = treino.history 
# "treino.history" retorna um contêiner de dicionário (uma generalização do conceito de lista): tipo "dict" 
# Um dicionário contém pares de (chave, valor)
treino_dic.keys()
# chaves no tf 2.1.0: ['loss', 'accuracy', 'val_loss', 'val_accuracy']

import matplotlib.pyplot as plt

font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }

acc = treino.history['accuracy']
val_acc = treino.history['val_accuracy']
loss = treino.history['loss']
val_loss = treino.history['val_loss']

epochs = range(len(acc))

plt.clf()

plt.plot(epochs, acc, 'b', label='Acurácia de treinamento')
plt.plot(epochs, val_acc, 'r', label='Acurácia de validação')
plt.title('Curvas de aprendizado', fontdict=font)
plt.xlabel('tempo (épocas)', fontdict=font)
plt.ylabel('Acurácia', fontdict=font)
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Erro de treinamento')
plt.plot(epochs, val_loss, 'r', label='Erro de validação')
plt.title('Curvas de aprendizado ', fontdict=font)
plt.xlabel('tempo (épocas)', fontdict=font)
plt.ylabel('Erro', fontdict=font)
plt.legend()

plt.show()

# avaliação do modelo
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Acurácia do teste:', test_acc)
# resultado esperado: por volta de 96% 

# previsões
predictions = model.predict(X_test)

%tensorboard --logdir logs/fit